In [3]:

# libraries
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import gc
from matplotlib import style
from pybaseball import statcast
import progressbar 
import warnings

pd.options.mode.chained_assignment = None  # default='warn'
warnings.simplefilter(action='ignore', category=FutureWarning)



### Load Statcast Data for Last Three Seasons (exclude 2020)

In [4]:

# 2022 season
sc_22 = statcast(start_dt='2022-04-07', end_dt='2022-10-5')

# 2021 season
sc_21 = statcast(start_dt='2021-04-01', end_dt='2021-10-3')

# 2019 season
sc_19 = statcast(start_dt='2019-03-28', end_dt='2019-09-29')

# merging data
df = pd.concat([sc_22, sc_21, sc_19], axis= 0)



This is a large query, it may take a moment to complete


/Users/loganmottley/opt/anaconda3/lib/python3.9/site-packages/pybaseball/statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)
100%|██████████| 182/182 [02:54<00:00,  1.04it/s]


This is a large query, it may take a moment to complete


/Users/loganmottley/opt/anaconda3/lib/python3.9/site-packages/pybaseball/statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)
100%|██████████| 186/186 [03:10<00:00,  1.02s/it]


This is a large query, it may take a moment to complete


/Users/loganmottley/opt/anaconda3/lib/python3.9/site-packages/pybaseball/statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)
100%|██████████| 186/186 [03:26<00:00,  1.11s/it]


### Calculating Run Expectancies

In [5]:


# selecting columns and droping duplicates se we can get a individual score after each inning
# removed 9th inning (http://www.tangotiger.net/re24.html)
re288_df = df[(df["inning"] < 9) & (df["balls"] < 4) & (df["strikes"] < 3) & (df["outs_when_up"] < 3)][["game_date", "game_pk", "at_bat_number", "pitch_number", "strikes", "balls", "on_1b", "on_2b", "on_3b", "inning", "inning_topbot", "away_score", "post_away_score", "home_score", "post_home_score", "outs_when_up", "description"]].iloc[::-1]

# cleaning up runner columns 
re288_df["on_1b"] = np.where(re288_df["on_1b"].notna() == True, "1", "_")
re288_df["on_2b"] = np.where(re288_df["on_2b"].notna() == True, "2", "_")
re288_df["on_3b"] = np.where(re288_df["on_3b"].notna() == True, "3", "_")

# making single runner column
re288_df["base_state"] = re288_df["on_1b"] + re288_df["on_2b"] + re288_df["on_3b"]

# making a runs scored event column 
re288_df["away_runs_scored"] = re288_df["post_away_score"] - re288_df["away_score"]
re288_df["home_runs_scored"] = re288_df["post_home_score"] - re288_df["home_score"]
re288_df['runs_scored'] = re288_df[["home_runs_scored", "away_runs_scored"]].sum(axis=1)

# making states column
re288_df["288_state"] = re288_df["base_state"].astype(str) + " | " + re288_df["outs_when_up"].astype(str) + " | " + re288_df["balls"].astype(str) + "-" + re288_df["strikes"].astype(str)

# claculate run expectancy for each state
state_names = re288_df["288_state"].unique().tolist()

# function to get run expectancy based on state
def get_re288(state): 
    
    # claculate run expectancy for each state
    x = re288_df[re288_df['288_state'].eq(state).groupby([re288_df.game_pk, re288_df.inning]).cummax()]
    x = x.groupby(["game_pk", "inning", "inning_topbot"])["runs_scored"].sum().reset_index()["runs_scored"].mean()

    return(x)

# for loop 
bar = progressbar.ProgressBar(maxval=len(state_names)).start()

re288_vals = [] 

for i in range(len(state_names)):

    re288_vals.append(get_re288(state_names[i]))
    # re288_vals = pd.DataFrame(pd.concat(result))
    bar.update(i)


# join 288 state names and run expectancies
re288_mat = pd.concat([pd.DataFrame(state_names).rename(columns={0 : "state"}), pd.DataFrame(re288_vals).rename(columns={0 : "run_expectancy"}).round(3)], axis=1)

# splitting state column into three
re288_mat[['base_state','outs', 'count']] = re288_mat['state'].str.split(' | ',expand=True).drop([1, 3], axis=1)

# dropping old state column
re288_mat = re288_mat.drop("state", axis = 1)

# RE288 for 0 outs
re288_mat_0 = re288_mat[re288_mat["outs"] == "0"]

# reordering counts
re288_mat_0['count'] = pd.Categorical(re288_mat_0['count'], ["0-2", "1-2", "0-1", "2-2", "1-1", "0-0", "1-0", "2-1", "3-2", "2-0", "3-1", "3-0"])
re288_mat_0 = re288_mat_0.sort_values("count")

# pivoting
re288_mat_0 = re288_mat_0.pivot_table(index=["base_state"], columns= ["count"], values= ["run_expectancy"]).reset_index().iloc[::-1]

# reordering base stats 
re288_mat_0['base_state'] = pd.Categorical(re288_mat_0['base_state'], ["___", "1__", "_2_", "12_", "__3", "1_3", "_23", "123"])

# sorting base states
re288_mat_0 = re288_mat_0.sort_values("base_state")

# table settings
cm = sns.color_palette("vlag", as_cmap=True)

# RE288 for 1 outs
re288_mat_1 = re288_mat[re288_mat["outs"] == "1"]

# reordering counts
re288_mat_1['count'] = pd.Categorical(re288_mat_1['count'], ["0-2", "1-2", "0-1", "2-2", "1-1", "0-0", "1-0", "2-1", "3-2", "2-0", "3-1", "3-0"])
re288_mat_1 = re288_mat_1.sort_values("count")

# pivoting
re288_mat_1 = re288_mat_1.pivot_table(index=["base_state"], columns= ["count"], values= ["run_expectancy"]).reset_index().iloc[::-1]

# reordering base stats 
re288_mat_1['base_state'] = pd.Categorical(re288_mat_1['base_state'], ["___", "1__", "_2_", "12_", "__3", "1_3", "_23", "123"])

# sorting base states
re288_mat_1 = re288_mat_1.sort_values("base_state")

# RE288 for 2 outs
re288_mat_2 = re288_mat[re288_mat["outs"] == "2"]

# reordering counts
re288_mat_2['count'] = pd.Categorical(re288_mat_2['count'], ["0-2", "1-2", "0-1", "2-2", "1-1", "0-0", "1-0", "2-1", "3-2", "2-0", "3-1", "3-0"])
re288_mat_2 = re288_mat_2.sort_values("count")

# pivoting
re288_mat_2 = re288_mat_2.pivot_table(index=["base_state"], columns= ["count"], values= ["run_expectancy"]).reset_index().iloc[::-1]

# reordering base stats 
re288_mat_2['base_state'] = pd.Categorical(re288_mat_2['base_state'], ["___", "1__", "_2_", "12_", "__3", "1_3", "_23", "123"])

# sorting base states
re288_mat_2 = re288_mat_2.sort_values("base_state")


### RE288 0 Outs

In [6]:


re288_mat_0.reset_index(drop=True).style.set_precision(2).background_gradient(cmap = cm, axis=None)




### RE288 1 Outs

In [7]:

re288_mat_1.reset_index(drop=True).style.set_precision(2).background_gradient(cmap = cm, axis=None)


### RE288 2 Outs

In [362]:

re288_mat_2.reset_index(drop=True).style.set_precision(2).background_gradient(cmap = cm, axis=None)
